In [1]:
using Pkg
Pkg.activate("Dev")
using Skylight
using BenchmarkTools

  Activating project at `~/repos/Skylight.jl/run/Dev`


In [4]:
spacetime = KerrSpacetimeBoyerLindquistCoordinates(M=1.0,a=0.9)
camera = ImagePlane(distance = 500.0,
                         observer_inclination_in_degrees = 90,
                         horizontal_side = 23.0,
                         vertical_side = 23.0,
                         horizontal_number_of_pixels = 200,
                         vertical_number_of_pixels = 200)
model = NovikovThorneDisk(inner_radius=6.0, outer_radius=18.0)
configurations = VacuumOTEConfigurations(spacetime=spacetime,
                                   camera = camera,
                                   radiative_model = model,
                                   unit_mass_in_solar_masses=1.0)
initial_data = initialize(configurations)
cb, cbp = callback_setup(configurations; rhorizon_bound=2e-1) #... or, define your own cb and cbp
run = integrate(initial_data, configurations, cb, cbp; method=VCABM(), reltol=1e-13, abstol=1e-21)
output_data = run.output_data

Equations integration stats:
Wall clock time: 70.159112087 seconds
Memory allocated: 16.125 GiB


8×40000 Matrix{Float64}:
 -1070.9         -1070.93        …  -1071.9         -1071.87
   551.163         551.163            551.163         551.163
     1.39638         1.39419            1.78948         1.78678
    -3.33518        -3.33584            3.34309         3.34271
    -1.00163        -1.00163           -1.00163        -1.00163
     0.997572        0.997576    …      0.997576        0.997571
    -3.70075e-5     -3.70045e-5         3.66757e-5      3.66806e-5
    -3.87576e-5     -3.83978e-5         3.90538e-5      3.94031e-5

In [5]:
#N=200 chunks 2
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=2)
nothing

In [ ]:
#N=200 chunks 1
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=1)
nothing

In [ ]:
#N=200 chunks 3
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=3)
nothing

In [ ]:
#N=50 chunks 2
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  316.410 μs (905 allocations: 123.83 KiB)


  7.151 μs (50 allocations: 45.25 KiB)


In [16]:
#N=50 chunks 3
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=3)
nothing

  316.190 μs (905 allocations: 123.83 KiB)


  12.428 μs (70 allocations: 47.91 KiB)


In [15]:
#N=50 chunks 1
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=1)
nothing

  316.049 μs (905 allocations: 123.83 KiB)


  5.442 μs (25 allocations: 41.92 KiB)


In [ ]:
#N=10 chunks 2
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  12.362 μs (41 allocations: 5.55 KiB)


  4.822 μs (48 allocations: 7.78 KiB)


In [ ]:
#N=3
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  975.571 ns (9 allocations: 1.05 KiB)


  2.845 μs (48 allocations: 6.28 KiB)
